# Lesson 4: Auto-merging Retrieval

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()




['AzureOpenAI', 'Bedrock', 'Embeddings', 'Feedback', 'GroundTruthAgreement', 'Huggingface', 'Langchain', 'LiteLLM', 'OpenAI', 'OptionalImports', 'REQUIREMENT_BEDROCK', 'REQUIREMENT_LITELLM', 'REQUIREMENT_OPENAI', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'embeddings', 'feedback', 'groundtruth', 'mod_feedback', 'prompts', 'provider', 'v2']
this is Dinesh Dir
['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'core', 'embeddings', 'indices', 'legacy', 'llms', 'readers']
this is Dinesh Llama_index


In [21]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./GQWr1LYS36vjseT1.pdf"]
).load_data()

In [22]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

27 

<class 'llama_index.core.schema.Document'>
Doc ID: 1cad11ea-62dc-4577-ac6e-745b35843c46
Text: 41394 Rev. A July 2017 Preventive  Maintenance  SIGMA SPECTRUM
INFUSION PUMP WITH  MASTER DRUG LIBRARY 35700BAX Pump Operating
Software  Version 6.02 and 6.05 For use with Master Drug Library
Version 6


## Auto-merging retrieval setup

In [24]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [26]:
from llama_index.core.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [27]:
nodes = node_parser.get_nodes_from_documents([document])

In [29]:
from llama_index.core.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

Press ON/OFF  to power on the Pump.
2. From the Select Care Area screen, press the options menu soft key (softwar e version 6.05) 
or Help  then the Options soft Key (software version 6.02).
3.


In [30]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

Contact MedinaTechTraining@baxter.com for more information.
Preventive Maintenance and Frequency
PM consists of:
■Scheduled inspections, testing, and performance evaluations
■Servicing any SIGMA Spectrum Infusion Pump th at fails any inspection, test, or performance 
evaluation
PM is required on:
■All Pumps in service (use)
■Any Pump suspected of being physically dam aged, dropped, or having fluid intrusion.
Recommended PM frequency:
■Pump: Annually
NOTE: A PM or network expiration due date may be entered in the BIOMED options. A “Due for 
Inspection” reminder will appear at power on followed by the programming screens. At 
power off, a “Due for Inspection” display will appear prior to Pump shutdown.
When performing an annual PM on SIGMA Spectrum  Infusion Systems, record the findings on the 
check sheet provided. See “Preventive Maintenance Check Sheet” on page 20.

2 Preventive MaintenanceManual 41394 SIGMA Spectrum Infusion System
Revision A Preventive Maintenance
When using the Ann

### Building the index

In [45]:
import llama_index.core.llms.llm
print(dir(llama_index.core.llms.llm))

['Any', 'BaseLLM', 'BaseLLMComponent', 'BaseModel', 'BaseOutputParser', 'BasePromptTemplate', 'CBEventType', 'Callable', 'ChainMap', 'ChatMessage', 'ChatResponseAsyncGen', 'ChatResponseGen', 'CompletionResponseAsyncGen', 'CompletionResponseGen', 'CompletionToPromptType', 'Dict', 'EventPayload', 'Field', 'InputKeys', 'LLM', 'LLMChatComponent', 'LLMCompleteComponent', 'LLMPredictEndEvent', 'LLMPredictStartEvent', 'LLMStructuredPredictEndEvent', 'LLMStructuredPredictStartEvent', 'List', 'MessageRole', 'MessagesToPromptType', 'Optional', 'OutputKeys', 'PromptTemplate', 'Protocol', 'PydanticProgramMode', 'QueryComponent', 'Sequence', 'StringableInput', 'TYPE_CHECKING', 'TokenAsyncGen', 'TokenGen', 'ToolSelection', 'Union', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'astream_chat_response_to_tokens', 'astream_completion_response_to_tokens', 'default_completion_to_prompt', 'dispatcher', 'generic_messages_to_prompt', 'get_args', 'i

In [46]:

      
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [49]:
from llama_index.core.service_context import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

ImportError: `llama-index-embeddings-huggingface` package not found, please run `pip install llama-index-embeddings-huggingface`

In [44]:
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [45]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


### Defining the retriever and running the query engine

In [46]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [80]:
auto_merging_response = auto_merging_engine.query(
    "what are 20 steps for Air Detection test for Baxter Infusion pump?"
)

In [81]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** 1. Load a primed IV set into the Pump for testing.
2. Select any Care Area and press OK.
3. Press OK to view the Drug Library.
4. Press the arrow soft key to select BASIC and press OK.
5. Select Primary Bag and press OK.
6. For Mode select mL/hr and press OK.
7. Install a new approved IV set and repeat the test.
8. Set the Rate to 100 mL/hr and press OK.
9. Set VTBI mL to 50 mL and press OK.
10. Press RUN/STOP to start the infusion.
11. Press the yes soft key to proceed to the Infusion Running screen.
12. Introduce a 2.5 cm (1 in) minimum air bubble into the IV set upstream from the Pump.
13. Enter the Biomed name and date of the test.
14. Perform the inspections, tests, and performance evaluations described in this chapter.
15. Record the test result findings.
16. Return any Pump to Baxter that fails any inspection or test.
17. Verify that the Pump displays a DOWNSTREAM OCCLUSION alarm at a pressure within the 89 kPa ±41 kPa (48 kPa to 131 kPa) [13 psi ±6 psi (7 psi to 19 psi)] occlusion trip pressure setting.
18. Remove the pressure gauge and confirm that the Pump auto restarts.
19. Press RUN/STOP to stop the infusion.
20. In the event of a test failure, confirm the failure by repeating the test with a new (unused) IV set.

## Putting it all Together

In [17]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [18]:
from llama_index.llms import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)


In [19]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

## TruLens Evaluation

In [20]:
from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Two layers

In [21]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

In [22]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [23]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [24]:
eval_questions = []
with open('generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [25]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [26]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: 9525fab2-c6fa-4747-abb1-afb10dee3dd0.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: 3536f75f-0155-40af-b4c9-f96fefe247e4.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: 44d51f87-4268-4181-b661-0d6240aac726.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: f283f2a1-202e-4902-bafc-860698df9601.
> Parent node text: PAGE 23Each project is only one step on a longer journey, hopefully one that has a positive impac...

> Merging 1 nodes into parent node.
> Parent node id: 83ea225a-9550-4122-9441-31787098981f.
> Parent node text: PAGE 29If you’re preparing to sw

In [27]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.982609,0.419565,0.895747,12.875,0.003712


In [28]:
Tru().run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-31-2-34p56402.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Three layers

In [29]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048,512,128],
)

In [30]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)


In [31]:
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [32]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

> Merging 4 nodes into parent node.
> Parent node id: 4c1abf4b-86e0-41eb-8b96-0304e524c540.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 5 nodes into parent node.
> Parent node id: 4c27e4a1-e0b9-4f76-98a9-42e566be74b9.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: be742202-2ce3-44f6-80a3-2eca2e4725e1.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: 6240920e-7a53-4066-979f-12574bf66efd.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 5 nodes into parent node.
> Parent node id: 1f88bf72-b993-4c44-9f4d-86bbe47d1aa8.
> Parent node text: PAGE 27There’s a lot we don’t kn

In [33]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_1,0.986364,0.406444,1.149120,12.875,0.001906
app_0,0.983333,0.408333,0.890268,12.875,0.003712


In [34]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-31-2-34p56402.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>